In [1]:
import pandas as pd
import re
from collections import namedtuple
import pdfplumber
import os 
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
import numpy as np

### 1. Importing and  merging pop files
### 2. Adding Metropolitans and Micropolitans

In [2]:
pop_2010_2019 = pd.read_excel('../data/raw_data/TN/pop_est_2010-2019_tn.xlsx',sheet_name='Sheet1')
pop_2010_2019 = pd.DataFrame(pop_2010_2019)
pop_2010_2019 = pop_2010_2019.drop(['Census','Estimates Base',2010,2011,2012],axis=1)
pop_2010_2019.head()

,County,2013,2014,2015,2016,2017,2018,2019
0,Anderson,75299,75157,75456,75528,76056,76287,76978
1,Bedford,45565,46251,46948,47442,48211,49146,49713
2,Benton,16360,16197,16220,16082,15993,16246,16160
3,Bledsoe,13910,14501,14601,14738,14895,14883,15064
4,Blount,124874,125847,126954,128264,129999,131331,133088


In [3]:
pop_2020_2021 = pd.read_excel('../data/raw_data/TN/pop_est_2020-2021_tn.xlsx',sheet_name='Sheet1')
pop_2020_2021 = pd.DataFrame(pop_2020_2021)
pop_2020_2021 = pop_2020_2021.drop('Estimates Base',axis=1)
pop_2020_2021.head()

,County,2020,2021
0,Anderson,77209,77576
1,Bedford,50302,51119
2,Benton,15812,15872
3,Bledsoe,14930,15234
4,Blount,135566,137605


In [4]:
pop_combined = pd.merge(pop_2010_2019,pop_2020_2021,how="left",on="County")
pop_combined['County'] = pop_combined['County'].str.strip()
pop_combined['County'] = pop_combined['County'].str.upper()
pop_combined.tail(20)

,County,2013,2014,2015,2016,2017,2018,2019,2020,2021
75,SCOTT,22062,21990,21893,21890,21964,22028,22068,21842,21917
76,SEQUATCHIE,14537,14634,14641,14754,14767,14891,15026,15863,16396
77,SEVIER,93477,94660,95417,96470,97307,97895,98250,98497,99517
78,SHELBY,938363,937500,937067,936021,935251,936365,937166,928644,924454
79,SMITH,19117,19097,19308,19526,19732,19979,20157,19939,20172
80,STEWART,13277,13210,13187,13190,13422,13619,13715,13683,13855
81,SULLIVAN,156185,156404,156199,156182,156924,157599,158348,158092,159265
82,SUMNER,168754,172269,175326,179332,183756,187490,191283,197272,200557
83,TIPTON,61572,61596,61533,61223,61305,61576,61599,60961,61004
84,TROUSDALE,7810,7998,8051,9955,10831,11033,11284,11661,12035


In [5]:
metros = pd.read_excel('../data/raw_data/metro_areas.xlsx',sheet_name='Filtered_States')
metros_df = pd.DataFrame(metros)
metros_tn = metros_df[metros_df['State Name']=='Tennessee']
metros_tn['County'] = metros_tn['County'].str.strip()
metros_tn['County'] = metros_tn['County'].str.upper()
metros_tn.head()

C:\Users\iulia\AppData\Local\Temp/ipykernel_27104/3293348550.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  metros_tn['County'] = metros_tn['County'].str.strip()
C:\Users\iulia\AppData\Local\Temp/ipykernel_27104/3293348550.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  metros_tn['County'] = metros_tn['County'].str.upper()


,CBSA Title,Metropolitan/Micropolitan Statistical Area,County,State Name
4,"Knoxville, TN",Metropolitan Statistical Area,ANDERSON,Tennessee
22,"Shelbyville, TN",Micropolitan Statistical Area,BEDFORD,Tennessee
28,"Knoxville, TN",Metropolitan Statistical Area,BLOUNT,Tennessee
30,"Cleveland, TN",Metropolitan Statistical Area,BRADLEY,Tennessee
55,"Knoxville, TN",Metropolitan Statistical Area,CAMPBELL,Tennessee


In [6]:
file ='../data/raw_data/TN/TN_2010_2022.pdf'

In [7]:
pdf = pdfplumber.open(file)
table = pdf.pages[43].extract_table()
text = pdf.pages[43].extract_text()
table = table [1:]
last_ele = table[-1]
for ele in last_ele:
     match = re.search("(Total\s\s\d{4})",ele)
     boolean = bool(match)
     year = re.findall("(?<=Total\s\s)\d{4}",ele)
     if boolean ==True:
        print(year,"Found!")

['2020'] Found!


### Extracting data for years 2013 thru 2020 and combining with pop along with Metropolitans and Micropolitans

In [8]:
pages = list(range(12,44))
tn_df = pd.DataFrame(columns=['County','fatalities','injuries','PDO','total_crashes'])
pdf = pdfplumber.open(file)
for page in pages:            
    tbl = pdf.pages[page].extract_table()
    last_ele = tbl[-1]
    df = pd.DataFrame(tbl[1::],columns=['County','fatalities','injuries','PDO','total_crashes'])
    tn_df = tn_df.append(df)    
    for ele in last_ele:
        match = re.search("(Total\s\s\d{4})",ele)
        boolean = bool(match)
        year = re.findall("(?<=Total\s\s)\d{4}",ele)
        for var in year:
             name = var         
        if boolean == True:
            name = int(name)
            tn_df = tn_df.drop(['PDO'],axis=1)
            tn_df = tn_df.iloc[:-1,:]
            tn_df['County'] = tn_df['County'].str.strip()
            tn_df['County'] = tn_df['County'].str.upper()
            df_combined_tn = pd.merge(tn_df,pop_combined[['County', name]],on="County",how='left')
            df_combined_tn_new = pd.merge(df_combined_tn,metros_tn[['CBSA Title','Metropolitan/Micropolitan Statistical Area','County']],on="County",how='left')
            df_combined_tn_new.rename(columns = {name:'population'}, inplace = True)
            df_combined_tn_new.insert(0,'Year',name)
            df_combined_tn_new.to_excel('../data/cleaned_data/%s_TN.xlsx' % name, engine='xlsxwriter',index=False)
            tn_df = pd.DataFrame(columns=['County','fatalities','injuries','PDO','total_crashes'])
        else :
            continue

### Combining all the files into a master file

In [15]:
data_file_folder ='../data/cleaned_data/'
df = []
for file in os.listdir(data_file_folder):
        if file.endswith('_TN.xlsx'):             
             df.append(pd.read_excel(os.path.join(data_file_folder,file),sheet_name='Sheet1'))
df_tn_master = pd.concat(df, axis = 0)

In [16]:
df_tn_master.isna().sum()

Year                                            0
County                                          0
fatalities                                      0
injuries                                        0
total_crashes                                   0
population                                      2
CBSA Title                                    250
Metropolitan/Micropolitan Statistical Area    250
dtype: int64

In [17]:
df_tn_master.insert(1,'State','Tennessee') 
df_tn_master.head(10)

,Year,State,County,fatalities,injuries,total_crashes,population,CBSA Title,Metropolitan/Micropolitan Statistical Area
0,2013,Tennessee,ANDERSON,10,445,"1,769",75299.0,"Knoxville, TN",Metropolitan Statistical Area
1,2013,Tennessee,BEDFORD,13,287,987,45565.0,"Shelbyville, TN",Micropolitan Statistical Area
2,2013,Tennessee,BENTON,10,94,304,16360.0,NaN,NaN
3,2013,Tennessee,BLEDSOE,0,31,80,13910.0,NaN,NaN
4,2013,Tennessee,BLOUNT,24,734,"2,981",124874.0,"Knoxville, TN",Metropolitan Statistical Area
5,2013,Tennessee,BRADLEY,9,608,"2,895",101778.0,"Cleveland, TN",Metropolitan Statistical Area
6,2013,Tennessee,CAMPBELL,9,269,895,40247.0,"Knoxville, TN",Metropolitan Statistical Area
7,2013,Tennessee,CANNON,3,72,265,13700.0,"Nashville-Davidson--Murfreesboro--Franklin, TN",Metropolitan Statistical Area
8,2013,Tennessee,CARROLL,9,116,392,28606.0,NaN,NaN
9,2013,Tennessee,CARTER,6,267,"1,204",56978.0,"Johnson City, TN",Metropolitan Statistical Area


In [18]:
df_tn_master.tail(10)

,Year,State,County,fatalities,injuries,total_crashes,population,CBSA Title,Metropolitan/Micropolitan Statistical Area
85,2020,Tennessee,UNICOI,3,74,290,17867.0,"Johnson City, TN",Metropolitan Statistical Area
86,2020,Tennessee,UNION,9,59,187,19820.0,"Knoxville, TN",Metropolitan Statistical Area
87,2020,Tennessee,VANBUREN,0,20,97,6171.0,NaN,NaN
88,2020,Tennessee,WARREN,6,171,757,41023.0,"McMinnville, TN",Micropolitan Statistical Area
89,2020,Tennessee,WASHINGTON,8,580,"3,508",133080.0,"Johnson City, TN",Metropolitan Statistical Area
90,2020,Tennessee,WAYNE,2,54,250,16202.0,NaN,NaN
91,2020,Tennessee,WEAKLEY,9,110,440,32864.0,"Martin, TN",Micropolitan Statistical Area
92,2020,Tennessee,WHITE,4,94,427,27424.0,NaN,NaN
93,2020,Tennessee,WILLIAMSON,16,719,"3,853",249424.0,"Nashville-Davidson--Murfreesboro--Franklin, TN",Metropolitan Statistical Area
94,2020,Tennessee,WILSON,16,798,"3,250",148564.0,"Nashville-Davidson--Murfreesboro--Franklin, TN",Metropolitan Statistical Area


In [19]:
df_tn_master.to_excel('../data/TN.xlsx', engine='xlsxwriter',index=False)

In [21]:
df_tn_master_pop_gr = df_tn_master.groupby(["CBSA Title","Year","Metropolitan/Micropolitan Statistical Area"])["population"].sum().reset_index()
df_tn_master_pop_gr

,CBSA Title,Year,Metropolitan/Micropolitan Statistical Area,population
0,"Athens, TN",2013,Micropolitan Statistical Area,52431.0
1,"Athens, TN",2014,Micropolitan Statistical Area,52675.0
2,"Athens, TN",2015,Micropolitan Statistical Area,52502.0
3,"Athens, TN",2016,Micropolitan Statistical Area,52700.0
4,"Athens, TN",2017,Micropolitan Statistical Area,52931.0
5,"Athens, TN",2018,Micropolitan Statistical Area,53336.0
6,"Athens, TN",2019,Micropolitan Statistical Area,53794.0
7,"Athens, TN",2020,Micropolitan Statistical Area,53315.0
8,"Brownsville, TN",2013,Micropolitan Statistical Area,18234.0
9,"Brownsville, TN",2014,Micropolitan Statistical Area,18235.0


In [22]:
df_tn_master_pop_gr = pd.pivot(df_tn_master_pop_gr,index=["CBSA Title","Metropolitan/Micropolitan Statistical Area"],columns=["Year"])
df_tn_master_pop_gr

population                                                                             
Year                                                                                            2013       2014       2015       2016       2017       2018       2019       2020
CBSA Title                                     Metropolitan/Micropolitan Statistical Area                                                                                        
Athens, TN                                     Micropolitan Statistical Area                 52431.0    52675.0    52502.0    52700.0    52931.0    53336.0    53794.0    53315.0
Brownsville, TN                                Micropolitan Statistical Area                 18234.0    18235.0    18036.0    17822.0    17607.0    17347.0    17304.0    17795.0
Chattanooga, TN-GA                             Metropolitan Statistical Area                391913.0   393768.0   396728.0   400915.0   404205.0   407782.0   411737.0   411688.0
Clarksville, TN-KY                             Metropolitan Statistical Area                197529.0   202215.0   205585.0   207614.0   213193.0   218931.0   222708.0   234843.0
Cleveland, TN                                  Metropolitan Statistical Area                118410.0   119455.0   120543.0   121184.0   122180.0   123968.0   124942.0   126310.0
Cookeville, TN                                 Micropolitan Statistical Area                107625.0   108405.0   108675.0   109560.0   110999.0   112476.0   114272.0   114187.0
Crossville, TN                                 Micropolitan Statistical Area                 57438.0    57924.0    58217.0    58580.0    59023.0    59738.0    60520.0    61272.0
Dayton, TN                                     Micropolitan Statistical Area                 32529.0    32623.0    32373.0    32474.0    32671.0    32911.0    33167.0    32903.0
Dyersburg, TN                                  Micropolitan Statistical Area                 38093.0    37807.0    37753.0    37542.0    37339.0    37286.0    37159.0    36646.0
Greeneville, TN                                Micropolitan Statistical Area                 68336.0    68464.0    68586.0    68529.0    68821.0    69164.0    69069.0    70220.0
Jackson, TN                                    Metropolitan Statistical Area                179836.0   179358.0   178607.0   178306.0   178336.0   178315.0   178644.0   180383.0
Johnson City, TN                               Metropolitan Statistical Area                200470.0   200021.0   200331.0   201457.0   201909.0   202900.0   203649.0   207262.0
Kingsport-Bristol, TN-VA                       Metropolitan Statistical Area                212881.0   212885.0   212569.0   212747.0   213544.0   214312.0   215134.0   214761.0
Knoxville, TN                                  Metropolitan Statistical Area                829017.0   832928.0   837818.0   844942.0   853645.0   861234.0   869046.0   881628.0
Lawrenceburg, TN                               Micropolitan Statistical Area                 42028.0    42304.0    42592.0    43056.0    43388.0    43771.0    44142.0    44189.0
Lewisburg, TN                                  Micropolitan Statistical Area                 31140.0    31287.0    31590.0    32035.0    33020.0    33806.0    34375.0    34441.0
Martin, TN                                     Micropolitan Statistical Area                 34155.0    34001.0    33872.0    33610.0    33325.0    33413.0    33328.0    32864.0
McMinnville, TN                                Micropolitan Statistical Area                 39875.0    39983.0    40262.0    40428.0    40717.0    40826.0    41277.0    41023.0
Memphis, TN-MS-AR                              Metropolitan Statistical Area               1038747.0  1038219.0  1037880.0  1036947.0  1036706.0  1038497.0  1039898.0  1031744.0
Morristown, TN                                 Metropolitan Statistical Area                137924.0   138280.0   139212.0   139924.0   140898.0   141778.0   142749.0   14283

In [23]:
df_tn_master_pop_analysis = df_tn_master_pop_gr.pct_change(axis='columns')
df_tn_master_pop_analysis

population                                                                      
Year                                                                                            2013      2014      2015      2016      2017      2018      2019      2020
CBSA Title                                     Metropolitan/Micropolitan Statistical Area                                                                                 
Athens, TN                                     Micropolitan Statistical Area                     NaN  0.004654 -0.003284  0.003771  0.004383  0.007651  0.008587 -0.008904
Brownsville, TN                                Micropolitan Statistical Area                     NaN  0.000055 -0.010913 -0.011865 -0.012064 -0.014767 -0.002479  0.028375
Chattanooga, TN-GA                             Metropolitan Statistical Area                     NaN  0.004733  0.007517  0.010554  0.008206  0.008849  0.009699 -0.000119
Clarksville, TN-KY                             Metropolitan Statistical Area                     NaN  0.023723  0.016665  0.009869  0.026872  0.026915  0.017252  0.054488
Cleveland, TN                                  Metropolitan Statistical Area                     NaN  0.008825  0.009108  0.005318  0.008219  0.014634  0.007857  0.010949
Cookeville, TN                                 Micropolitan Statistical Area                     NaN  0.007247  0.002491  0.008144  0.013134  0.013306  0.015968 -0.000744
Crossville, TN                                 Micropolitan Statistical Area                     NaN  0.008461  0.005058  0.006235  0.007562  0.012114  0.013090  0.012426
Dayton, TN                                     Micropolitan Statistical Area                     NaN  0.002890 -0.007663  0.003120  0.006066  0.007346  0.007779 -0.007960
Dyersburg, TN                                  Micropolitan Statistical Area                     NaN -0.007508 -0.001428 -0.005589 -0.005407 -0.001419 -0.003406 -0.013806
Greeneville, TN                                Micropolitan Statistical Area                     NaN  0.001873  0.001782 -0.000831  0.004261  0.004984 -0.001374  0.016664
Jackson, TN                                    Metropolitan Statistical Area                     NaN -0.002658 -0.004187 -0.001685  0.000168 -0.000118  0.001845  0.009734
Johnson City, TN                               Metropolitan Statistical Area                     NaN -0.002240  0.001550  0.005621  0.002244  0.004908  0.003691  0.017741
Kingsport-Bristol, TN-VA                       Metropolitan Statistical Area                     NaN  0.000019 -0.001484  0.000837  0.003746  0.003596  0.003836 -0.001734
Knoxville, TN                                  Metropolitan Statistical Area                     NaN  0.004718  0.005871  0.008503  0.010300  0.008890  0.009071  0.014478
Lawrenceburg, TN                               Micropolitan Statistical Area                     NaN  0.006567  0.006808  0.010894  0.007711  0.008827  0.008476  0.001065
Lewisburg, TN                                  Micropolitan Statistical Area                     NaN  0.004721  0.009685  0.014087  0.030748  0.023804  0.016831  0.001920
Martin, TN                                     Micropolitan Statistical Area                     NaN -0.004509 -0.003794 -0.007735 -0.008480  0.002641 -0.002544 -0.013922
McMinnville, TN                                Micropolitan Statistical Area                     NaN  0.002708  0.006978  0.004123  0.007149  0.002677  0.011047 -0.006154
Memphis, TN-MS-AR                              Metropolitan Statistical Area                     NaN -0.000508 -0.000327 -0.000899 -0.000232  0.001728  0.001349 -0.007841
Morristown, TN                                 Metropolitan Statistical Area                     NaN  0.002581  0.006740  0.005115  0.006961  0.006246  0.006849  0.000567
Nashville-Davidson--Murfreesboro--Franklin, TN Metropolitan Statistical Area                     NaN  0.019215  0.020539  0.020293  0.018042  0.016117  0.015